In [ ]:
#r "nuget: SqlSugarCore"

using SqlSugar;
using SqlSugar.DbConvert;

Installed Packages SqlSugarCore, 5.1.4.172

In [ ]:
#nullable enable

/// <summary>
/// 日志等级
/// </summary>
public enum LogLevel
{
    Debug = 0,
    Info = 10,
    Warning = 20,
    Error = 30
}

/// <summary>
/// 日志结构定义，这里为了可以使用ORM框架存入数据库，因此需要增加SugarSQL特性
/// </summary>
/// <remarks>
/// 1. 按日分表，设置分表的名字模板，3个变量必须要有，这么设计为了兼容开始按年，后面改成按月
/// 2. SplitField：分表字段 在插入的时候会根据这个字段插入哪个表，在更新删除的时候用这个字段找出相关表
/// </remarks>
[SplitTable(SplitType.Day)]
[SugarTable("Log_{year}{month}{day}")]
public class Log
{
    public Log() { }

    /// <summary>
    /// 日志时间
    /// </summary>
    [SplitField]
    public DateTime LogTime { get; set; }

    /// <summary>
    /// 日志等级，显示为Enum字符串
    /// </summary>
    [SugarColumn(ColumnDataType = "varchar(20)", SqlParameterDbType = typeof(EnumToStringConvert))]
    public LogLevel LogLevel { get; set; }

    /// <summary>
    /// 日志标题
    /// </summary>
    [SugarColumn(ColumnDataType = "varchar(255)")]
    public string LogMessage { get; set; } = "";

    /// <summary>
    /// 日志细节
    /// </summary>
    [SugarColumn(ColumnDataType = "varchar(20000)")]
    public string LogDescription { get; set; } = "";

    [SugarColumn(IsJson = true)]
    public string[] LogDescriptionArray { get; set; } = [];


    public Log(LogLevel _level, string _message, string _description)
    {
        LogTime = DateTime.Now;
        LogLevel = _level;
        LogMessage = _message;
        // LogDescription = _description;
    }

    public override string ToString()
    {
        string _log = $"{LogTime:yyyy年MM月dd日 HH:mm:ss}  {LogLevel,-8}  {LogMessage}";
        return _log;
    }
}

In [ ]:


SqlSugarClient LogSqlConnection { get; set; }
LogSqlConnection = new SqlSugarClient(new ConnectionConfig()
{
    DbType = DbType.Sqlite,
    ConnectionString = $"DataSource=SQLiteTest.db",
    IsAutoCloseConnection = true,
    MoreSettings=new ConnMoreSettings() 
    {
        EnableCodeFirstUpdatePrecision= true,//启用
    }
});

if (!LogSqlConnection.Ado.IsValidConnection()) { throw new Exception("连接参数不存在"); }
LogSqlConnection.DbMaintenance.CreateDatabase();      // 创建数据库，如果已存在，则没有任何影响
LogSqlConnection.CodeFirst.InitTables(typeof(Log));

/// <summary>
/// 写入日志到数据库中
/// </summary>
/// <param name="_log"></param>
void WriteLogToDatabase(Log _log)
{
    if (LogSqlConnection is not null)
    {
        lock (LogSqlConnection)
        {
            LogSqlConnection.Insertable(_log).SplitTable().ExecuteCommand();
        }
    }
}

In [ ]:

WriteLogToDatabase(new Log(LogLevel.Info, "测试", "这是一个测试日志"){LogDescriptionArray = new string[]{"1", "2", "3"}});